# 2018-06-05 - Psychophysics using psychopy
On repasse sur un paradoxe plus classique de psychophysique.

On récupère les initiales :

In [1]:
from psychopy import visual, core, event
import MotionClouds as MC

if False:
    from psychopy import gui
    exp_name = "Perception de l'orientation"
    exp_info = {
                'Initiales ': ''
                }
    subj_info = gui.DlgFromDict(dictionary=exp_info, title=exp_name)
else:
    exp_info = {'Initiales ': 'LP'}
print (exp_info)

{'Initiales ': 'LP'}


On défini la génération de MotionClouds :

In [2]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = '/tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

On définit les paramètres et on teste :

In [3]:
from psychopy import visual, core, event
import MotionClouds as MC

test_length = 150  # number of trials
#trial_length = 2.6  # length of trial, in second
fixation_length = .5  # length of fixation, in second
stim1_length = 0.25  # length of first stim, in second
stim2_length = 0.25  # length of the second stim, in second

fullscr = False

fullscr = True

# Psychopy window
win = visual.Window(fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
mouse = event.Mouse(visible=False)

fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
fixation.autolog = False

MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
bitmap1 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap1.autolog = False
bitmap2 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap2.autolog = False
#bitmap3 = visual.ImageStim(win, '%s.png' %
#                           str(2 * (np.pi/3)), mask='circle')
#bitmap3.autolog = False


#msg = visual.TextStim(win, text='<- LEFT or RIGHT ->', color='black')
msg = visual.TextStim(win, text='?', color='black')

#shift_dict = {'right': 2,
#              'left':  3}


# Answer list
ans_list = []
N_B_thetas = 5
B_thetas = np.pi*np.logspace(-7, 0, N_B_thetas, base=2)
    
std_theta = np.pi/6
for trial in range(test_length):
    clock = core.Clock()
    fixation.draw()
    win.flip()
    
    # Shift to left (3) or to right (2) ?
    #shift = random.randint(2, 3)
    theta = std_theta * np.random.randn()

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    bitmap1.setImage(MC1)

    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2
    bitmap2.setImage(MC2)

    #MC3 = generate_random_cloud(2 * (np.pi/3), B_theta=B_theta)  # if shift = 3
    # MC to psychopy objects
    
    # Times the trial
    while clock.getTime() < fixation_length + stim1_length + stim2_length:
        if 0.0 <= clock.getTime() < fixation_length:  # fixation
            fixation.draw()
        if fixation_length + 0.01 <= clock.getTime() < fixation_length + stim1_length:
            bitmap1.draw()
        if fixation_length + stim1_length <= clock.getTime() < fixation_length + stim1_length + stim2_length:
            bitmap2.draw()
        win.flip()

    # display orientation choice
    msg.draw()
    win.flip()

    # answer using the keyboard
    while True:
        ans = event.waitKeys()

        if ans in ['escape', 'q']:
            win.close()
            core.quit()
        if len(ans) > 0:
            print(trial, theta*180/np.pi, ans[0], B_theta*180/np.pi)
            break

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    ans_list.append([trial, theta, ans[0], B_theta])

win.close()

0 19.421976096889782 right 18.88708098174817
1 -51.41344775455638 left 64.31149007430338
2 17.22943066783557 right 2.2956835769280906
3 12.012104890432758 right 8.621953085904861
4 -19.888685968227527 left 14.782234806666851
5 36.93831947829553 right 17.9837349534872
6 -7.857352718130218 left 9.05504482664442
7 13.039081003322497 right 82.17000588723204
8 -5.51942866180441 left 32.38167305052465
9 6.307551609450483 right 15.524765384211547
10 -1.1641514658871979 left 134.14139237788663
11 0.2844382023292694 right 14.075218560253347
12 -20.016937389596983 left 9.509891319935742
13 -6.383881563110162 left 27.954030581403764
14 -27.849771640729358 left 18.88708098174817
15 -26.15237392137278 left 180.0
16 -24.85695928260333 left 90.63234489451486
17 -2.465870753621579 left 58.306728400028334
18 -50.09674870471695 left 74.49779518963292
19 2.7954930914776894 right 134.14139237788663
20 -3.786426268147164 left 16.304594223203157
21 -28.5911411309091 left 6.425337733111642
22 -30.63445540601

In [4]:
import pickle
pickle.dump(ans_list, open('Psychophys_discrim_%s.p' %
                           exp_info['Initiales '], 'wb'))